In [60]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [61]:
import scipy.integrate as sp


In [62]:
# Data paths
train_dir = r"D:\studeyyy\dogImages\dogImages\train"
validation_dir = r"D:\studeyyy\dogImages\dogImages\valid"
test_dir = r"D:\studeyyy\dogImages\dogImages\test"

In [63]:
# Image dimensions
img_height = 224
img_width = 224
batch_size = 32

In [64]:
# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [65]:
validation_datagen = ImageDataGenerator(rescale=1./255)


In [66]:
# Load and prepare the training, validation, and test datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    class_mode='categorical')

Found 6680 images belonging to 133 classes.
Found 835 images belonging to 133 classes.


In [67]:
# Base model (pre-trained MobileNetV2)
base_model = MobileNetV2(weights='imagenet', include_top=False)

In [68]:
# Add custom head for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

In [69]:
# Combine base model and custom head
model = Model(inputs=base_model.input, outputs=predictions)

In [70]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

In [71]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [72]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)

NameError: name 'scipy' is not defined

In [ ]:
# Evaluate the model on test data
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print('Test accuracy:', test_acc)